# EDA

In [ ]:
print(df.shape)
# フィールド情報を統合したDataFrameを作成
field_info_df = pd.DataFrame({
    'dtype': df.dtypes,
    'isna_sum': df.isna().sum(),
    'nunique': df.nunique()
})

print("フィールド情報統合DataFrame:")
field_info_df.sort_values('nunique', ascending=True)

# テスト

In [ ]:
import pandas as pd
def compare_dataframes(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
) -> bool | pd.DataFrame:
    df1c = df1.copy()
    df2c = df2.copy()

    # NaNを持っているか
    if df1c.isna().any().any():
        print('df1にNaNがあります')
    if df2c.isna().any().any():
        print('df2にNaNがあります')

    # objectに変換
    df1c = df1c.fillna('').astype(str)
    df2c = df2c.fillna('').astype(str)

    # カラムの一致を確認（差分のみ表示）
    df1_columns = set(df1c.columns)
    df2_columns = set(df2c.columns)

    df1_only = sorted(df1_columns - df2_columns)
    df2_only = sorted(df2_columns - df1_columns)

    if df1_only or df2_only:
        parts = []
        if df1_only:
            parts.append(f'df1にのみ存在: {df1_only}')
        if df2_only:
            parts.append(f'df2にのみ存在: {df2_only}')
        print('カラム差分があります: ' + '; '.join(parts))
        return False
    print('カラムは同じです')

    common_columns = sorted(df1c.columns)
    df1c = df1c[common_columns]
    df2c = df2c[common_columns]

    # レコード内容の差分（種類の差）を確認
    df1dd = df1c.drop_duplicates(subset=common_columns)
    df2dd = df2c.drop_duplicates(subset=common_columns)
    df_merged = pd.merge(df1dd, df2dd, how='outer', indicator=True, on=common_columns)
    diff_records = df_merged[df_merged['_merge'] != 'both']
    if not diff_records.empty:
        print('レコード差分があります（値の種類が違う）')
        print(diff_records.shape)

        return diff_records
    print('レコードの種類は同じです')

    # 重複の分布（同一レコードの出現回数）が同じかを確認
    cnt1 = df1c.groupby(common_columns, dropna=False).size().rename('count').reset_index()
    cnt2 = df2c.groupby(common_columns, dropna=False).size().rename('count').reset_index()
    cnt_m = cnt1.merge(
        cnt2,
        on=common_columns,
        how='outer',
        indicator=True,
        suffixes=('_df1', '_df2'),
    ).fillna({'count_df1': 0, 'count_df2': 0})
    multiplicity_diff = cnt_m.query('count_df1 != count_df2')
    if not multiplicity_diff.empty:
        print('同一レコードの出現回数（重複の分布）に差分があります')

        return multiplicity_diff

    print('重複の分布も同じです')
    print('Same df')
    return True

# 有効数字関数

In [ ]:
from decimal import Decimal


def format_number_as_string(x: str, sig=3) -> str:
    try:
        if isinstance(x, str):
            s = x.strip()
            d = Decimal(s)
        else:
            return '0'
    except Exception:
        return '0'

    if d.is_zero():
        return '0'
    e = d.adjusted()  # 先頭有効桁の指数
    q = Decimal(f'1e{e - (sig - 1)}')  # 丸め単位
    r = d.quantize(q, rounding=ROUND_FLOOR)
    s = format(r, 'f')
    if '.' in s:
        s = s.rstrip('0').rstrip('.')
    return s